In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Configurações Iniciais
ano = 2025  # Definindo o ano base
nome_mercado = "MC Supermercados"

# --- 1. Definição das Datas Comemorativas (Feriados/Eventos Reais de 2025) ---
# Formato: (Nome do Evento, Data do Evento)
datas_comemorativas = [
    ("Carnaval", "2025-03-04"),       # Terça de Carnaval
    ("Páscoa", "2025-04-20"),         # Domingo de Páscoa
    ("Dia das Mães", "2025-05-11"),   # 2º Domingo de Maio
    ("Dia dos Namorados", "2025-06-12"), # Data Fixa
    ("Aniversário MC", "2025-07-02"), # Data Definida por nós
    ("Dia dos Pais", "2025-08-10"),   # 2º Domingo de Agosto
    ("Semana Farroupilha", "2025-09-20"), # Feriado RS
    ("Dia das Crianças", "2025-10-12"), # Data Fixa
    ("Halloween", "2025-10-31"),      # Data Fixa
    ("Black Friday", "2025-11-28"),   # Última sexta de Nov
    ("Natal e Fim de Ano", "2025-12-25") # Referência para o Natal
]

# --- 2. Função para Calcular Janela do Encarte ---
# Regra Padrão: Começa na Sexta anterior e termina no Domingo posterior (10 dias aprox)
def calcular_periodo_encarte(nome_evento, data_ref_str):
    data_ref = datetime.strptime(data_ref_str, "%Y-%m-%d")
    
    # Lógica Especial: Natal e Fim de Ano (Vai até 31/12)
    if nome_evento == "Natal e Fim de Ano":
        # Começa na sexta-feira antes do Natal
        # weekday(): 0=Seg, 4=Sex. Se dia 25 é Quinta(3), Sexta anterior é dia 19.
        offset_sexta = (data_ref.weekday() - 4) % 7
        if offset_sexta == 0: offset_sexta = 7 # Se já for sexta, pega a anterior
        
        data_inicio = data_ref - timedelta(days=offset_sexta)
        data_fim = datetime(data_ref.year, 12, 31) # Trava em 31/12
        return data_inicio, data_fim

    # Regra Geral (Sexta antes -> Domingo depois)
    # Encontrar a sexta-feira anterior à data do evento (ou a própria, se for sexta)
    # Nota: Se o evento é domingo (6), sexta é 6-4=2 dias antes.
    # Ajuste simples: recuar até achar a sexta-feira.
    days_to_friday = (data_ref.weekday() - 4) % 7
    # Se o evento cair numa sexta (ex: Black Friday), queremos começar ANTES? 
    # Geralmente encarte de Black Friday é a "Semana Black". Vamos manter a regra da sexta anterior.
    if days_to_friday == 0: 
        days_to_friday = 7 # Garante que volta 1 semana se cair na sexta
        
    data_inicio = data_ref - timedelta(days=days_to_friday)
    
    # Data Fim: Domingo após a data de referência (ou o próprio, se for domingo)
    days_to_sunday = (6 - data_ref.weekday()) % 7
    if days_to_sunday == 0 and data_ref.weekday() != 6: # Caso de borda
        days_to_sunday = 0
        
    data_fim = data_ref + timedelta(days=days_to_sunday)
    
    # Garante duração mínima de fim de semana (extensão pós evento)
    # Se o evento for na terça (Carnaval), o domingo seguinte é longe.
    # Se a lógica "Domingo depois" ficar muito longe (>10 dias totais), ajustamos?
    # Vamos manter simples: Sexta anterior até Domingo Posterior ao evento.
    
    return data_inicio, data_fim

# --- 3. Gerando a Lista de Promoções (Encartes) ---
lista_promocoes = []
id_counter = 1

for evento, data in datas_comemorativas:
    inicio, fim = calcular_periodo_encarte(evento, data)
    
    lista_promocoes.append({
        "id_promocao": id_counter,
        "nome_promocao": f"Encarte {evento}",
        "tipo_promocao": "Sazonal (Encarte)",
        "data_inicio": inicio.date(),
        "data_fim": fim.date(),
        "categoria_foco": "Mix Geral (10%)", # Regra de 10% aleatório
        "descricao": "Descontos em produtos selecionados + Decoração de loja"
    })
    id_counter += 1

# --- 4. Adicionando Promoções Semanais Recorrentes ---
# Como são recorrentes o ano todo, podemos representá-las de forma genérica
# ou criar uma entrada válida para o ano todo (01/01 a 31/12) com a regra de dia da semana.
ofertas_semanais = [
    ("Segunda da Limpeza", "Limpeza e Higiene", 0), # 0 = Segunda
    ("Terça da Padaria", "Padaria", 1),
    ("Quarta do Hortifruti", "Hortifruti", 2),
    ("Quinta do Açougue", "Açougue", 3),
    ("Sexta das Bebidas", "Bebidas", 4)
]

for nome, cat, dia_semana in ofertas_semanais:
    lista_promocoes.append({
        "id_promocao": id_counter,
        "nome_promocao": nome,
        "tipo_promocao": "Recorrente Semanal",
        "data_inicio": datetime(ano, 1, 1).date(),
        "data_fim": datetime(ano, 12, 31).date(),
        "categoria_foco": cat,
        "descricao": f"Oferta válida apenas às {['Segundas', 'Terças', 'Quartas', 'Quintas', 'Sextas'][dia_semana]}"
    })
    id_counter += 1

# --- 5. Criando o DataFrame ---
df_dim_promocoes = pd.DataFrame(lista_promocoes)

# Exibindo para verificação
print(f"--- Tabela dim_promocoes ({nome_mercado}) ---")
print(df_dim_promocoes[["id_promocao", "nome_promocao", "data_inicio", "data_fim", "tipo_promocao"]])

# Opcional: Salvar em CSV
df_dim_promocoes.to_csv("dim_promocoes.csv", index=False, encoding='utf-8-sig')

--- Tabela dim_promocoes (MC Supermercados) ---
    id_promocao               nome_promocao data_inicio    data_fim  \
0             1            Encarte Carnaval  2025-02-28  2025-03-09   
1             2              Encarte Páscoa  2025-04-18  2025-04-20   
2             3        Encarte Dia das Mães  2025-05-09  2025-05-11   
3             4   Encarte Dia dos Namorados  2025-06-06  2025-06-15   
4             5      Encarte Aniversário MC  2025-06-27  2025-07-06   
5             6        Encarte Dia dos Pais  2025-08-08  2025-08-10   
6             7  Encarte Semana Farroupilha  2025-09-19  2025-09-21   
7             8    Encarte Dia das Crianças  2025-10-10  2025-10-12   
8             9           Encarte Halloween  2025-10-24  2025-11-02   
9            10        Encarte Black Friday  2025-11-21  2025-11-30   
10           11  Encarte Natal e Fim de Ano  2025-12-19  2025-12-31   
11           12          Segunda da Limpeza  2025-01-01  2025-12-31   
12           13            Te